In [6]:
# pip install oracledb
# pip install sqlalchemy
### 위 2개 등록

import oracledb
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, ForeignKey, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import random

# 데이터베이스 연결 설정
dsn = "localhost/XE"
user = "kamp"
password = "kamp"

# SQLAlchemy 엔진 생성
engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")

# SQLAlchemy 세션 설정
Session = sessionmaker(bind=engine)
session = Session()

# SQLAlchemy 베이스 클래스 정의
Base = declarative_base()

# MemberData, GoCamping, MemberRating 테이블 정의
class MemberData(Base):
    __tablename__ = 'member_data'
    no_data = Column(Integer, primary_key=True)

class GoCamping(Base):
    __tablename__ = 'go_camping'
    content_id = Column(Integer, primary_key=True)

class MemberRating(Base):
    __tablename__ = 'member_rating'
    rate_idx = Column(Integer, primary_key=True)
    no_data = Column(Integer, ForeignKey('member_data.no_data'))
    content_id = Column(Integer, ForeignKey('go_camping.content_id'))
    rating = Column(Float)

    member = relationship('MemberData')
    content = relationship('GoCamping')

# 시퀀스를 사용하여 rate_idx 값을 생성하는 함수
def get_next_rate_idx():
    with engine.connect() as conn:
        result = conn.execute(text("SELECT rate_seq.NEXTVAL FROM dual"))
        return result.scalar()

# 데이터 조회
members = session.query(MemberData).filter(MemberData.no_data >= 2, MemberData.no_data <= 9).all()
contents = session.query(GoCamping).all()

# 각 멤버에 대해 2000개의 고유한 content_id를 선택하고 랜덤 점수 부여
ratings = []
for member in members:
    if len(contents) < 2000:     ## 생성할 평점 갯수(멤버당)
        raise ValueError("Not enough unique content_id available to assign 2000 ratings per member.")
    
    selected_contents = random.sample(contents, 2000)     ## 숫자 편집(위와 동일하게)
    for content in selected_contents:
        rating = random.uniform(1, 5)     ## 1점 ~ 5점 까지
        rating = round(rating * 2) / 2.0  # 0.5 단위로 반올림
        rate_idx = get_next_rate_idx()  # 시퀀스를 사용하여 rate_idx 생성
        ratings.append(MemberRating(rate_idx=rate_idx, no_data=member.no_data, content_id=content.content_id, rating=rating))

# 데이터베이스에 삽입
session.bulk_save_objects(ratings)
session.commit()

print("Random ratings have been successfully added to the database.")


C:\Users\Jung\AppData\Local\Temp\ipykernel_28316\891439832.py:21: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Random ratings have been successfully added to the database.
